In [1]:
import json, os
from datetime import datetime

from tqdm import tqdm
import numpy as np
import pickle
from matplotlib import pyplot as plt

import nest_asyncio
import asyncio
nest_asyncio.apply()

from sentence_transformers import SentenceTransformer
from transformers import pipeline
import psutil

from data.models import DrugLabel, ProductSection
from data.util import compute_section_embedding

/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Some M1 ARM architecture issues with transformers / tokenizers
# Ended up installing django-extensions and running Jupyter with shell-plus from the venv
# Tried to override Django settings as Jupyter was using the Docker networking, so `postgres` wasn't resolving, but this didn't work
# Added /etc/hosts mapping `127.0.0.1 postgres` so this would work on my local, see https://github.com/instructure/lti_tool_provider_example/issues/4
# See: https://gist.github.com/EtsuNDmA/dd8949061783bf593706559374c8f635
# See: https://stackoverflow.com/questions/61926359/django-synchronousonlyoperation-you-cannot-call-this-from-an-async-context-u

# os.environ["DJANGO_SETTINGS_MODULE"] = "dle.settings"
# os.environ.get("DJANGO_SETTINGS_MODULE")
# import os, sys
# import django
# PROJECTPATH = '/Users/colecrawford/GitHub/dle'
# sys.path.insert(0, PROJECTPATH)
# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings")
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # https://docs.djangoproject.com/en/4.1/topics/async/#async-safety
# os.chdir(PROJECTPATH)
# django.setup()

In [2]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [8]:
sections = ProductSection.objects.filter(label_product__drug_label__source="TGA").all()
print(sections.count())
subset = sections[0:500]
print(subset.count())

22875
500


In [9]:
pubmedbert_model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')

In [10]:
start = datetime.now()

for section in tqdm(subset):
    section.bert_vector = json.dumps(compute_section_embedding(section.section_text, model=pubmedbert_model, normalize=True))
    section.save()
    
end = datetime.now()
elapsed = end - start
print(f" ------------- vectorized {subset.count()} sections in { int(elapsed.total_seconds()) } seconds")
print(f"{int(elapsed.total_seconds()) / subset.count() } seconds per section")

  0%|                                                                                                                                                                                                                                                                | 0/500 [00:00<?, ?it/s]2023-04-02 18:55:32,902 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed for 1 times in a row, putting on 1 second timeout
2023-04-02 18:55:32,903 WARNING Retrying request after failure (attempt 0 of 3)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_transport.py", line 329, in perform_request
    meta, raw_data = node.perform_request(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_node/_http_urllib3.py", line 199, in perform_request
    raise err from None
elastic_transport.ConnectionError: 

2023-04-02 18:55:33,895 ERROR Error handling 'post_save' signal for ProductSection object (957974)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/apps.py", line 143, in _update_search_index
    instance.index_search_document(index=index)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/models.py", line 401, in index_search_document
    _ = get_client().index(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/utils.py", line 414, in wrapped
    return api(*args, **kwargs)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py", line 2264, in index
    return self.perform_request(  # type: ignore[

2023-04-02 18:55:36,331 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed for 2 times in a row, putting on 2 second timeout
2023-04-02 18:55:36,332 WARNING Retrying request after failure (attempt 1 of 3)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_transport.py", line 329, in perform_request
    meta, raw_data = node.perform_request(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_node/_http_urllib3.py", line 199, in perform_request
    raise err from None
elastic_transport.ConnectionError: Connection error caused by: NewConnectionError(<urllib3.connection.HTTPSConnection object at 0x28c8f3af0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known)
2023-04-02 18:55:36,334 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed

  1%|██▍                                                                                                                                                                                                                                                     | 5/500 [00:04<07:00,  1.18it/s]2023-04-02 18:55:36,750 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed for 1 times in a row, putting on 1 second timeout
2023-04-02 18:55:36,750 WARNING Retrying request after failure (attempt 0 of 3)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_transport.py", line 329, in perform_request
    meta, raw_data = node.perform_request(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_node/_http_urllib3.py", line 199, in perform_request
    raise err from None
elastic_transport.ConnectionError: 

2023-04-02 18:55:36,952 ERROR Error handling 'post_save' signal for ProductSection object (957973)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/apps.py", line 143, in _update_search_index
    instance.index_search_document(index=index)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/models.py", line 401, in index_search_document
    _ = get_client().index(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/utils.py", line 414, in wrapped
    return api(*args, **kwargs)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py", line 2264, in index
    return self.perform_request(  # type: ignore[

...

2023-04-02 18:58:41,117 ERROR Error handling 'post_save' signal for ProductSection object (958464)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/apps.py", line 143, in _update_search_index
    instance.index_search_document(index=index)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/models.py", line 401, in index_search_document
    _ = get_client().index(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/utils.py", line 414, in wrapped
    return api(*args, **kwargs)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py", line 2264, in index
    return self.perform_request(  # type: ignore[

2023-04-02 18:58:42,706 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed for 2 times in a row, putting on 2 second timeout
2023-04-02 18:58:42,706 WARNING Retrying request after failure (attempt 1 of 3)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_transport.py", line 329, in perform_request
    meta, raw_data = node.perform_request(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_node/_http_urllib3.py", line 199, in perform_request
    raise err from None
elastic_transport.ConnectionError: Connection error caused by: NewConnectionError(<urllib3.connection.HTTPSConnection object at 0x2cf149ba0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known)
2023-04-02 18:58:42,709 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 498/500 [03:12<00:01,  1.05it/s]2023-04-02 18:58:44,698 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed for 1 times in a row, putting on 1 second timeout
2023-04-02 18:58:44,698 WARNING Retrying request after failure (attempt 0 of 3)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_transport.py", line 329, in perform_request
    meta, raw_data = node.perform_request(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_node/_http_urllib3.py", line 199, in perform_request
    raise err from None
elastic_transport.ConnectionError: 

2023-04-02 18:58:45,004 ERROR Error handling 'post_save' signal for ProductSection object (958471)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/apps.py", line 143, in _update_search_index
    instance.index_search_document(index=index)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch_django/models.py", line 401, in index_search_document
    _ = get_client().index(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/utils.py", line 414, in wrapped
    return api(*args, **kwargs)
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py", line 2264, in index
    return self.perform_request(  # type: ignore[

 ------------- vectorized 500 sections in 193 seconds
0.386 seconds per section


In [12]:
from elasticsearch_django.settings import get_client
es = get_client()

In [14]:
vectorized = ProductSection.objects.filter(label_product__drug_label__source="TGA").filter(bert_vector__isnull=False)
for section in tqdm(vectorized):
    es.index(index="productsection", document=section.as_search_document(), id=section.id)

  0%|                                                                                                                                                                                                                                                                | 0/500 [00:00<?, ?it/s]2023-04-02 19:00:03,820 WARNING Node <Urllib3HttpNode(https://es01:9200)> has failed for 1 times in a row, putting on 1 second timeout
2023-04-02 19:00:03,821 WARNING Retrying request after failure (attempt 0 of 3)
Traceback (most recent call last):
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_transport.py", line 329, in perform_request
    meta, raw_data = node.perform_request(
  File "/Users/colecrawford/GitHub/dle/docs/section-matching/caleb-data-scripts-venv/lib/python3.10/site-packages/elastic_transport/_node/_http_urllib3.py", line 199, in perform_request
    raise err from None
elastic_transport.ConnectionError: 

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPSConnection object at 0x1757bba90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known))

In [ ]:
list(model.encode("brain bleeding"))

In [ ]:
data_file_fda = 'data/output/human-rx-openfda-drug.json'
RUN_DIAGNOSTIC = False

In [ ]:
with open(data_file_fda) as f:
    data_fda = json.load(f)

In [ ]:
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')

In [ ]:
keys = list(data_fda.keys())
print(len(keys))
k = keys[0]
print(data_fda[k].keys())
print(data_fda[k]['metadata'].keys())

sample_keys = keys[0:500]

In [ ]:
keys_fda, drugs_fda = zip(*data_fda.items())
sections_fda = [d['Label Text'].keys() for d in drugs_fda]
sections_fda = sorted(set([s for slist in sections_fda for s in slist]))

In [ ]:
sections_fda

In [ ]:
def compute_section_embedding(text, word_count=256):
    n_segments = 1 + len(text.split()) // word_count
    print(n_segments)
    vecs = np.zeros((n_segments,768))
    for i in range(n_segments):
        segment = text.split()[ (i)*word_count : (i+1)*word_count ]
        print(segment)
        vecs[i,:] = model.encode( ' '.join(segment) )
    return np.mean(vecs, axis=0)

In [ ]:
!pip install ray

In [ ]:
num_cpus = psutil.cpu_count(logical=True)
print('Number of available CPUs:', num_cpus)

ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

pipe = pipeline(task = 'feature-extraction', model='pritamdeka/S-PubMedBert-MS-MARCO', batch_size=1, device=-1)

pipe_id = ray.put(pipe)

@ray.remote
def vectorize(pipeline, 

In [ ]:
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def compute_vector_wrapper(key):
    drug = data_fda[key]
    sections = drug['Label Text']
    print(key)
    vectors[key] = {}
    for k,v in sections.items():
        # { "4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9" = { "spl_product_data_elements": <VECTOR> } }
        if len(v) > 1:
            # print(f"{key} - {k} - {len(v)} subsections")
            # print(" ".join(v))
            vectors[key][k] = compute_section_embedding(" ".join(v))
        else:
            vectors[key][k] = compute_section_embedding(v[0])

vectors = {}
start = datetime.now()
loop = asyncio.get_event_loop()
looper = asyncio.gather(*[compute_vector_wrapper(key) for key in sample_keys])
results = loop.run_until_complete(looper)
end = datetime.now()
elapsed = end - start

total_sections = 0
for key in sample_keys:
    total_sections += len(data_fda[key]['Label Text'])
print(f"fin ------------- { int(elapsed.total_seconds()) } seconds")
print(f"{len(sample_keys)} drug labels processed: { int(elapsed.total_seconds()) / len(sample_keys) } seconds per drug")
print(f"{total_sections} sections processed: { int(elapsed.total_seconds()) / total_sections } seconds per section")

In [ ]:
# AYSNCIO Tests
# 100 drug labels processed: 2.85 seconds per drug
# 2223 sections processed: 0.1282051282051282 seconds per section

# 500 drug labels processed: 3.202 seconds per drug
# 11593 sections processed: 0.13810057793496075 seconds per section

In [ ]:
vectors

In [ ]:
vectors = {}
section_count = 0
start = datetime.now()
for key in tqdm.tqdm(sample_keys):
    drug = data_fda[key]
    sections = drug['Label Text']
    # print(key)
    vectors[key] = {}
    for k,v in sections.items():
        # { "4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9" = { "spl_product_data_elements": <VECTOR> } }
        if len(v) > 1:
            # print(f"{key} - {k} - {len(v)} subsections")
            # print(" ".join(v))
            vectors[key][k] = compute_section_embedding(" ".join(v))
        else:
            vectors[key][k] = compute_section_embedding(v[0])
            section_count += 1
            
end = datetime.now()
elapsed = end - start

print(f"{len(sample_keys)} drug labels processed: { int(elapsed.total_seconds()) / len(sample_keys) } seconds per drug")
print(f"{section_count} sections processed: { int(elapsed.total_seconds()) / section_count } seconds per section")

In [ ]:
from multiprocessing import Pool
import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

def compute_vector_wrapper(key):
    drug = data_fda[key]
    sections = drug['Label Text']
    vectors[key] = {}
    for k,v in sections.items():
        # { "4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9" = { "spl_product_data_elements": <VECTOR> } }
        if len(v) > 1:
            # print(f"{key} - {k} - {len(v)} subsections")
            # print(" ".join(v))
            vectors[key][k] = compute_section_embedding(" ".join(v))
        else:
            vectors[key][k] = compute_section_embedding(v[0])
    print(f"{key} completed", flush=True)

with Pool(4) as pool:
     tqdm(pool.imap(compute_vector_wrapper, sample_keys))

In [ ]:
print(section_count)

In [ ]:
len(vectors['4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9']['spl_product_data_elements'])

In [ ]:
test_vector = vectors['4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9']['spl_product_data_elements']
test_vector_list = test_vector.tolist()
json_vector = json.dumps(test_vector_list)
json.loads(json_vector)

In [ ]:
print(f"num subsections: {len(data_fda['ca8bfc0a-d43b-1072-e053-2995a90a66f0']['Label Text']['warnings'])}")
data_fda['ca8bfc0a-d43b-1072-e053-2995a90a66f0']['Label Text']['warnings']

In [ ]:
data_fda['ca8bfc0a-d43b-1072-e053-2995a90a66f0']

In [ ]:
type(vectors['4fdf3b7e-f6ff-4d78-a928-c9d47c5bc9d9']['spl_product_data_elements'])

In [ ]:
!pip list